In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
from pathlib import Path
import pandas as pd

In [31]:
checkpoints = os.listdir('./checkpoints')
checkpoints

['baseline-epoch=68.ckpt',
 'baseline-val_metric=0.51015-epoch=49.ckpt',
 'baseline-val_metric=0.20308-epoch=0.ckpt',
 'baseline-epoch=0.ckpt']

In [44]:
import segmentation_models_pytorch as smp
import torch 

checkpoint = './checkpoints/baseline-val_metric=0.51015-epoch=49.ckpt'

state_dict = torch.load(checkpoint)['state_dict']
for key in list(state_dict.keys()):
    if key.startswith('model.'):
        state_dict[key[6:]] = state_dict.pop(key)

model = smp.Unet(encoder_name='resnet18', encoder_weights="imagenet", in_channels=9, classes=1)
model.load_state_dict(state_dict)
model.cpu().eval();
# model

In [45]:
traced = torch.jit.trace(model, torch.randn(10, 9, 256, 256))
# traced

/home/juan/.local/lib/python3.10/site-packages/segmentation_models_pytorch/base/model.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h % output_stride != 0 or w % output_stride != 0:


In [46]:
path = Path('/fastdata/contrails')

records = os.listdir(path / 'test')
len(records)

2

In [47]:
stats = pd.read_csv(path/'stats.csv', index_col=0)
stats

,min,max,mean,std
8,175.82391,280.14868,233.67686,4.545741
9,180.74695,279.48690,242.25447,6.057177
10,181.44263,331.13394,250.75069,7.620164
11,179.33739,332.06036,274.41205,13.668153
12,187.69131,306.21823,255.52716,8.708825
13,179.34510,338.05670,276.60184,14.446373
14,178.71164,338.63330,275.35940,14.736154
15,178.36511,333.21048,272.56410,14.367307
16,137.39153,311.97977,260.42580,11.049050


In [48]:
import torch 
import numpy as np

class Dataset(torch.utils.data.Dataset):
    def __init__(self, records):
        self.records = records

    def __len__(self):
        return len(self.records)
    
    def preprocess(self, record):
        bands = range(8,17)
        data = []
        for band in bands:
            image = np.load(path / 'test' / record / f'band_{band:02d}.npy')
            image = (image - stats.loc[band]['mean']) / stats.loc[band]['std']
            data.append(image)
        data = np.stack(data, axis=-1)
        return data[...,4,:]
    
    def __getitem__(self, ix):
        record = self.records[ix]
        image = self.preprocess(record)
        return record, torch.from_numpy(image).permute(2,0,1)

In [49]:
ds = Dataset(records)
dl = torch.utils.data.DataLoader(ds, batch_size=8, num_workers=4, pin_memory=True)

In [50]:
def rle_encode(x, fg_val=1):
    """
    Args:
        x:  numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encoding as list
    """

    dots = np.where(
        x.T.flatten() == fg_val)[0]  # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def list_to_string(x):
    """
    Converts list to a string representation
    Empty list returns '-'
    """
    if x: # non-empty list
        s = str(x).replace("[", "").replace("]", "").replace(",", "")
    else:
        s = '-'
    return s

In [51]:
submission = {'record_id': [],	'encoded_pixels': []}

In [52]:
traced.eval()
traced.cuda()
with torch.no_grad():
    for records, x in dl:
        y_hat = traced(x.cuda())
        masks = y_hat.sigmoid().cpu().numpy() > 0.5
        masks = masks.astype(np.int32)
        for ix in range(masks.shape[0]):
            mask = masks[ix][0]
            rle = rle_encode(mask)
            record = records[ix]
            submission['record_id'].append(record)
            if len(rle) == 0: 
                submission['encoded_pixels'].append('-')
            else:
                submission['encoded_pixels'].append(' '.join(map(str,rle)))

In [53]:
submission = pd.DataFrame(submission)
submission.to_csv('submission.csv', index=False)
submission

,record_id,encoded_pixels
0,1002653297254493116,-
1,1000834164244036115,-


In [54]:
torch.save(traced, 'model.pt')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_146175/4057463644.py:1 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_146175/4057463644.py'                       │
│                                                                                                  │
│ /home/juan/.local/lib/python3.10/site-packages/torch/serialization.py:441 in save                │
│                                                                                                  │
│    438 │                                                                                         │
│    439 │   if _use_new_zipfile_serialization:                                                    │
│    440 │   │   with _open_zipfile_writer(f) as opened_zipfile:                                   │
│ ❱  441 │   │   │   _save(obj, opened_zipfile, pickle_module, pickle_protocol)                    │
│    442 │   │   │   return                                                                        │
│    443 │   else:                                                                                 │
│    444 │   │   with _open_file_like(f, 'wb') as opened_file:                                     │
│                                                                                                  │
│ /home/juan/.local/lib/python3.10/site-packages/torch/serialization.py:653 in _save               │
│                                                                                                  │
│    650 │   data_buf = io.BytesIO()                                                               │
│    651 │   pickler = pickle_module.Pickler(data_buf, protocol=pickle_protocol)                   │
│    652 │   pickler.persistent_id = persistent_id                                                 │
│ ❱  653 │   pickler.dump(obj)                                                                     │
│    654 │   data_value = data_buf.getvalue()                                                      │
│    655 │   zip_file.write_record('data.pkl', data_value, len(data_value))                        │
│    656                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Tried to serialize object 
__torch__.segmentation_models_pytorch.decoders.unet.model.___torch_mangle_1307.Unet which does not have a 
__getstate__ method defined!